# Write data transformations using sklearn pipeline

In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
class DataGrabber(BaseEstimator, TransformerMixin):
  """
  Returns the different feature names
  """
  def __init__(self, url):
    self.url = url
    pass

  def fit(self, X, y):
    return self
  
  def transform(self, _):
    return pd.read_excel(self.url)

In [3]:
class ColumnRenamer(BaseEstimator, TransformerMixin):
  """
  Renames dict columns
  """
  def __init__(self, columns):
    self.columns = columns
    pass

  def fit(self, X, y):
    return self
  
  def transform(self, df):
    return df.rename(columns=self.columns)

In [9]:
class ColumnReformatter(BaseEstimator, TransformerMixin):
  """
  Renames dict columns
  """
  def __init__(self, columns_funs):
    self.columns_funs = columns_funs
    pass

  def fit(self, X, y):
    return self
  
  def transform(self, df):
    for column, fun in self.columns_funs.items():
      df[column] = df[column].apply(fun)
    return df

In [31]:
class CandidateFlatMapper(BaseEstimator, TransformerMixin):
  """
  Flatmaps a row with n candidates to n rows
  """
  def __init__(self):
    pass

  def fit(self, X, y):
    return self
  
  def _flatten(self, df_x):
    d = {'candidate_party': [], 'votes': []}
    tups = [("BName{0}".format(i), "Tally{0}".format(i)) for i in range(1,11)]
    for name, tally in tups:
        d['candidate_party'].extend(df_x[name])
        d['votes'].extend(df_x[tally])
    return pd.DataFrame(d)
  
  def transform(self, df):
    out = df.groupby(["county", "precinct", "office", "district", "Candidate Group"])\
      .apply(self._flatten).reset_index()
    out = out.dropna(subset=["candidate_party"])
    return out

In [53]:
import re

class CandidatePartySplitter(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.writein_cand = re.compile(r"^\.\s(Write-in)\s-\s(.*)")
    self.cand_party = re.compile(r"^\.\s(.*)\s-\s([A-Z]?)$")
    self.cand = re.compile(r"^\.\s(.*)$")
    pass

  def fit(self, X, y):
    return self
  
  def get_candidate_party(self, s):
    if not isinstance(s, str):
      print(s)
      print(type(s))
      raise ValueError("Unrecognized input format")
      
    m_wi = self.writein_cand.match(s)
    m_cp = self.cand_party.match(s)
    m_c  = self.cand.match(s)
    if m_wi is not None:
      party, name = m_wi.groups()
      return (name, party)
    elif m_cp is not None:
      name, party = m_cp.groups()
      return (name, party)
    elif m_c is not None:
      name = m_c.groups()[0]
      return (name, "")
    else:
      print(s)
      raise ValueError("Unrecognized input string format")
  
  def transform(self, df):
    df["candidate"], df["party"] = zip(*df.candidate_party.map(self.get_candidate_party))
    return df

In [107]:
from nameparser import HumanName

class CandidateToCategorical(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  
  def fit(self, X, y):
    return self

  def name_mapper(self, s):
    name = HumanName(s)
    return "{last}{suffix}, {first}".format(
      **dict(last=name.last, suffix=name.suffix, first=name.first)
    )
  
  def transform(self, df):
    cands = pd.Series(df.candidate.unique())
    categories = cands.loc[cands.map(self.name_mapper).sort_values().index]
    df["candidate"] = pd.Categorical(df.candidate, 
                                     categories=categories).map(lambda s: s.replace(",",""))
    return df

In [108]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("grab_data", DataGrabber("http://share.tn.gov/sos/election/results/2010-11/November2010.xls")),
    ("rename_columns", ColumnRenamer({
            "District":"district",
            "COUNTY":"county",
            "OFFICENAME":"office",
            "Precinct Sequence":"precinct"
          })
    ),
    ("reformat_columns", ColumnReformatter({"county": lambda s: s.capitalize()})),
    ("rectify_candidates", CandidateFlatMapper()),
    ("split_candidate_party", CandidatePartySplitter()),
    ("categorical_candidates", CandidateToCategorical()),
  ])

In [109]:
df = pipe.transform(pd.DataFrame())

In [110]:
df.sort_values("candidate")

,county,precinct,office,district,Candidate Group,level_5,candidate_party,votes,candidate,party
50471,Sullivan,1,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,132,Joe Mike Akard,D
50561,Sullivan,5,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,13,Joe Mike Akard,D
50591,Sullivan,6,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,1124,Joe Mike Akard,D
50641,Sullivan,8,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,64,Joe Mike Akard,D
50671,Sullivan,9,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,177,Joe Mike Akard,D
50691,Sullivan,10,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,491,Joe Mike Akard,D
50991,Sullivan,22,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,272,Joe Mike Akard,D
51011,Sullivan,23,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,123,Joe Mike Akard,D
51041,Sullivan,24,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,10,Joe Mike Akard,D
51091,Sullivan,26,Tennessee House of Representatives District,3.0,1,1,. Joe Mike Akard - D,224,Joe Mike Akard,D


In [112]:
for office in df.office.unique():
  df.loc[df.office == office, :].groupby(["county", "candidate", "party", "office", "district"]).agg({
      "votes": np.sum,
    }).reset_index().sort_values(["candidate", "county"])\
  .to_csv(re.sub(r"\s+", "_", office)+".csv", index=False)

In [114]:
df.loc[df.candidate == "D.H. (Andy) Andrew",:]

,county,precinct,office,district,Candidate Group,level_5,candidate_party,votes,candidate,party
1682,Blount,1,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,13,D.H. (Andy) Andrew,I
1702,Blount,2,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,10,D.H. (Andy) Andrew,I
1722,Blount,3,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,5,D.H. (Andy) Andrew,I
1742,Blount,4,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,6,D.H. (Andy) Andrew,I
1762,Blount,5,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,12,D.H. (Andy) Andrew,I
1782,Blount,6,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,6,D.H. (Andy) Andrew,I
1802,Blount,7,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,3,D.H. (Andy) Andrew,I
1832,Blount,8,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,3,D.H. (Andy) Andrew,I
1852,Blount,9,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,5,D.H. (Andy) Andrew,I
1872,Blount,10,U. S. House of Representatives District,2.0,1,2,. D.H. (Andy) Andrew - I,12,D.H. (Andy) Andrew,I
